In [26]:
from moviepy import VideoFileClip
from pathlib import Path
import speech_recognition as sr
from yt_dlp import YoutubeDL
from pprint import pprint
from PIL import Image
import matplotlib.pyplot as plt

In [7]:
import os

OPEN_API_KEY = os.getenv("OPEN_API_KEY")
os.environ["OPEN_API_KEY"] = OPEN_API_KEY

print(os.getcwd())

c:\Users\csing\VSCode\Projects\MultiModal_RAG_Video-Processing


In [24]:
video_url = "https://youtu.be/sw_IK4M7S0A?si=FT_ZFohHnGmzaMHp"

In [9]:
output_video_path = "./video_data/"
output_folder = "./mixed_data/"
output_audio_path = "./mixed_data/output_audio.wav"

In [10]:
filepath = output_video_path + "input_vid.mp4"
Path(output_folder).mkdir(parents=True, exist_ok=True)

In [30]:
def download_video(url, output_path):
    options = {
        'outtmpl': f'{output_path}/input_vid.%(ext)s',
        'format': 'best[ext=mp4]/best',  # Prefer mp4 format to avoid merging
        'quiet': True,
        'no_warnings': True,
    }

    with YoutubeDL(options) as ydl:
        info = ydl.extract_info(url, download=True)
        metadata = {
            "Author": info.get("uploader"),
            "Title": info.get("title"),
            "Views": info.get("view_count"),
            "Duration": info.get("duration"),
        }
        return metadata

In [12]:
def video_to_images(video_path, output_folder):
    clip = VideoFileClip(video_path)
    clip.write_images_sequence(os.path.join(output_folder, "frame_%04d.png"), fps=1)
    

In [13]:
def video_to_audio(video_path, output_audio_path):
    clip = VideoFileClip(video_path)
    audio = clip.audio
    audio.write_audiofile(output_audio_path)

In [14]:
def audio_to_text(audio_path):
    recognizer = sr.Recognizer()
    audio=sr.AudioFile(audio_path)

    with audio as source:
        audio_data = recognizer.record(source)

        try:
            text = recognizer.recognize_whisper(audio_data)
        
        except sr.UnknownValueError:
            print("Audio not recognized")         
    return text

In [29]:
metadata_vid = download_video(video_url, output_video_path)

In [31]:
video_to_images(filepath, output_folder)

In [32]:
video_to_audio(filepath, output_audio_path)

MoviePy - Writing audio in ./mixed_data/output_audio.wav


MoviePy - Done.


In [34]:
text_data = audio_to_text(output_audio_path)

In [35]:
text_data

" If you're building AI apps that use custom data, choosing the right framework can save you weeks of development time. In 2025, two top contenders are Langchain and Lama Index. Both are powerful, but they serve slightly different needs. Let's break it down. First, let's talk about ease of use. Lama Index is known for being beginner-friendly. It focuses on helping you connect LLMs with your data. Think PDFs, databases, or notion pages. Using simple Python code. Reviews on GitHub and G2 highlight how easy it is to get started. Even for solo developers or startups. Langchain, on the other hand, is more flexible, but also more complex. It offers chains, agents, tools, and memory systems to build dynamic apps. Developers love the modularity, but reviews on Reddit and TrustPilot note the steep learning curve and sometimes inconsistent documentation. Now, let's talk features and customization. Langchain is built for advanced use cases. Multi-step reasoning, agent workflows, and integration w

In [36]:
with open(os.path.join(output_folder, "text_data.txt"), "w") as f:
    f.write(text_data)
    print("Text data saved to text_data.txt")
    f.close()

Text data saved to text_data.txt


In [37]:
os.remove(output_audio_path)
print("Audio file removed after processing.")

Audio file removed after processing.
